In [71]:
import serial
import time
import pandas as pd
import scipy
from scipy import signal
import numpy as np
import math
from statistics import mean

In [72]:
import pickle
global clf
clf = pickle.load(open('testmodel.pkl','rb'))

In [693]:
def event_detection(data):
    '''
    Detects devices being turned On
    Returns the timestamp indices where devices are turned on
    '''
    data = scipy.signal.medfilt(data, kernel_size=None) #remove any error spikes
    dbydx = np.gradient(data) # find edges
#     plt.plot(dbydx);
    on_events = [] # maybe convert to numpy?
    for idx,x in enumerate(dbydx):
        if x > THRSHLD:
            on_events.append(idx)
    for x in on_events:
        for y in range(x+1,x+91): # to remove false turn on signatures, give delay for transition
            if y in on_events:
                on_events.remove(y)
    return on_events

In [694]:
def get_signal (length):
    '''
    Takes the length of signature required, (two cases : full length, reconstruct)
    returns signature and timestamps
    
    '''
#     start = time.time() 
    count = 0
    buffer = np.array([])
    time_buffer = np.array([])
    while count!= length :
        b = ser.readline()         # read a byte string
        string_n = b.decode(errors='ignore')  # decode byte string into Unicode  
        try:
            val, timestamp = string_n.split(sep = ',')
            val = float(val) #convert string to float
#             timestamp = pd.to_datetime(timestamp) # maybe move string to timestamp conversion to time span calculation code
            buffer = np.append(buffer, val) # helps getting data in seconds
            time_buffer = np.append(time_buffer,timestamp)
            count += 1
        except:
            print('read failed, retrying!') # serial comm error
    print(string_n)
#     print(f'{time.time()-start:.3} s') 
    return buffer,time_buffer

In [695]:
def predict_clf(data): #unused
    global clf
    guess = clf.predict(data)
    acc = clf.predict_proba(data).max()
    print(f'Device turned on at {sig_ts} and is probably {guess}, Acc.: {acc}') 

In [696]:
def log_event(name,sig_ts,init_rise,rise,sig):
    '''
    Stores the following:
    1. Signature of appliance,
    2. Time-stamp of turning ON
    3. Name
    4. Rise in consumption
    
    '''
    global logs_df
    global prev_mean #simple case
    logs_df = logs_df.append(pd.DataFrame({'Sig':[sig],
                                           'Name':name,
                                           'Turned ON':pd.to_datetime(sig_ts),
                                           'Init Rise':init_rise,
                                           'Total Rise':rise,
                                           'Fall':prev_mean, # causing problem i guess
                                           'Turned OFF':0,
                                           'Flag':0},index=[0]),ignore_index=True)

#finding cumulative rise !!!
arr = logs_df[logs_df['Flag']==0].index.values # gets on devices indices
cumul_rise = 0
for x in arr:
    cumul_rise = cumul_rise + logs_df['Init Rise'][x] # cumultaive rise of all active appliances
if math.isclose(total_rise,cumul_rise,abs_tol=abs(x*1/100)):
    logs_df.loc[[num],['Turned OFF']] = pd.to_datetime(ts)
    print(f'Appliance number {num} turned off at {ts}')
    logs_df.loc[[num],['Flag']] = 1

In [743]:
def track_device(num,val,ts):
    #simple case
    global logs_df
    global active_appliances
    start = time.time()
    if logs_df['Flag'][num] == 0: #to check if the appliance is still ON 
        nm = logs_df['Name'][num]
        print(f' {nm} is being tested for being turned OFF') # works fine
        total_rise = logs_df['Total Rise'][num] #steady state vallur or current value
        init_rise = logs_df['Init Rise'][num] #redundant??
        fall = logs_df['Fall'][num] #redundant?
        offset = total_rise - val # gives current rise or fall 
        total_rise = total_rise - offset # while calculating remember turning off a device makes the remaining value of the left device
        logs_df.loc[[num],['Total Rise']] = total_rise
        if (logs_df['Flag']==0).sum() > 1: # number of active appliances are greater than 1
            arr_actv_indices = logs_df[logs_df['Flag']==0].index.values # gets on devices indices
            arr_actv_indices = np.delete(arr_actv_indices, np.where(arr_actv_indices == num)) # remove this device's rise
            print(f'number of active devices excluding {nm }are {len(arr_actv_indices)}')
            cumul_rise = 0
            for x in arr_actv_indices: # sum all other appliance's rise
                cumul_rise = cumul_rise + logs_df['Init Rise'][x] # cumultaive rise of all active appliances
                print(f'Cumulative Rise = {cumul_rise}')
            if math.isclose(total_rise,cumul_rise,abs_tol=abs(cumul_rise*1/100)): #is current rise close to cumultaive rise
                print('cumulative sum was found remaining')
                logs_df.loc[[num],['Turned OFF']] = pd.to_datetime(ts)
                print(f'{nm} turned off at {ts}')
                logs_df.loc[[num],['Flag']] = 1
#                 print(f'time taken to track down {time.time() - start}')
                return 1
                      
            # not sure about this one, shold tell for plus one active devie only
            
            for idx, x in enumerate(logs_df['Init Rise']): # used not condition because reverse turning of events caused problem
                if math.isclose(total_rise,x,abs_tol=abs(x*1/100)): #if more than two appliances are there, then might have to add their individual rises and check.
                    print('Found the value to be same!!!!')
                    return 0
                else:
                    print('value not same')
            return 0
#                     logs_df.loc[[idx],['Turned OFF']] = pd.to_datetime(ts)
#                     print(f'Appliance number {idx} turned off at {ts}')
#                     logs_df.loc[[idx],['Flag']] = 1
#                     print(f'time taken to track down 2 {time.time() - start}')
#                     return 1
#                 else:
#                     return 0
        else: # if only this appliance is on
            if math.isclose(total_rise,0,abs_tol=abs(init_rise*1/100)): #fall caused problem, replaced with zero, however actually there would be a phantom consumption so might replace with that!!! ??
                logs_df.loc[[num],['Turned OFF']] = pd.to_datetime(ts)
                print(f'Appliance number {num} turned off at {ts}')
                logs_df.loc[[num],['Flag']] = 1
#                 print(f'time taken to track down 3{time.time() - start}')
    #             active_appliances -= 1
                return 1
            else:
                return 0
    else: #if the appliance was not on
        return 0

In [735]:
global logs_df
logs_df = pd.DataFrame([],columns = ['Sig','Name','Turned ON','Init Rise','Total Rise','Fall','Turned OFF','Flag'])

In [736]:
SIG_SIZE = 90 # 1 sec contains 30 samples ; 3 seconds 90 samples.
THRSHLD = 5 # Rise in kWH to be detected for classification
BUFFER_SIZE = 180 # signal buffer size i.e 6 seconds (1s has 30 samples)
i = 1 #test purpose iterator
global prev_mean
prev_mean = 0 
# These are for creating local database and saving signatures!!!
prev_guess=''
global active_appliances
active_appliances = 0

In [738]:
ser = serial.Serial('COM3', 115200) # remove later
time.sleep(2)


# Read and record the data
for i in range(100):
    buffer, time_buffer = get_signal(BUFFER_SIZE) # 6s seconds data into buffer
    events = event_detection(buffer) # if no on events, check for off events
    if len(events)>0: # events detected ; can remove this if statement because its redundant 
#         start = time.time()
#         print(f'{len(events)} events detected') #works fine
        for ev in events:
            sig = buffer[ev:ev+SIG_SIZE]
            if len(sig) != SIG_SIZE: #if signal is not enough reconstruct by getting new data
#                 print('signal not adequate, shifting buffer') #works fine
                buffer = np.roll(buffer, -SIG_SIZE) 
                time_buffer = np.roll(time_buffer, -SIG_SIZE)
                temp_buffer, temp_time_buffer = get_signal(SIG_SIZE)
                buffer[BUFFER_SIZE-SIG_SIZE:] = temp_buffer
                time_buffer[BUFFER_SIZE-SIG_SIZE:] = temp_time_buffer
                events = event_detection(buffer) # recheck events to get new indices, later requires breaking outer for loop
                print(f'{len(events)} events detected on reconstructing')
            if len(events)>0:
                for ev in events:
                    sig = buffer[ev:ev+SIG_SIZE] # extracting signature from buffer
                    if sig.mean() > prev_mean: # signifies a rise in signal
                        sig = sig - prev_mean # remove previous bias
                        sig_ts = time_buffer[ev] #extract time stamp as well
                        sig = sig.reshape((sig.shape[0]//SIG_SIZE,SIG_SIZE)) # reshape for classifier
#                         rise = buffer[ev+SIG_SIZE:].mean() # data to log , maybe put rise as steady state value
                        temp_buffer, _ = get_signal(90)
                        rise = temp_buffer.mean() #to get steady state rise
                        init_rise = rise - prev_mean
#                         prev_mean = rise #causing problem of false turn off
                        guess = clf.predict(sig) # predict
                        end = time.time() # for timing information
                        acc = clf.predict_proba(sig).max() #check accuracy
#                         print(f'{guess} turned on at {sig_ts} with rise {rise}, Acc.: {acc}')
                        if acc > 0.80: 
                            if prev_guess == guess:
                                temp_name = logs_df['Name'].to_numpy()[-1]
                                print(f'It seems the {temp_name} was turned on again at {sig_ts}. ') 
                                print(f'It probably has multiple cycles. If true retype {temp_name}, else name it\n')
                                name = input('Enter the name!\n') # to save the signaure with user defined name for future classification
                                if name == logs_df['Name'].to_numpy()[-1]:
                                    # for clubbing
                                    # save signature
                                    pass
                                else:
                                    # save signature
                                    pass
                                prev_guess = guess #redundant may remove
                                log_event(name,sig_ts,init_rise,rise,sig) #store data
                                #here might want to decrement active_devices and remove start time from it and club it by comparing prev and current guess!
                            else:
                                name = input(f'An appliance  was detected at {sig_ts}! Some people named it as {guess}, would you like to name it?\n')
                                prev_guess = guess
                                log_event(name,sig_ts,init_rise,rise,sig)
                        else:
                            name = input(f'An unrecognized device was detected at {sig_ts}, it seems {guess} but not sure, so please name it\n')
#                             name = guess
                            prev_guess = guess
                            log_event(name,sig_ts,init_rise,rise,sig)
                active_appliances += 1
#                 print(f'Total time taken to identify the appliance {end-start:.3f} s')
                break # breaks previous 'for-loops' of events 
        

    else:
        prev_mean = buffer.mean() #keep tracking the signal level
        for aplnc in range(active_appliances): #if there are active devices
            # update values of rise to find off event
            curr_val = prev_mean #redundant
            m = track_device(aplnc,curr_val,time_buffer[-1])
            if m == 1:
                mm,ss = divmod((pd.to_datetime(logs_df['Turned OFF'][aplnc]) - pd.to_datetime(logs_df['Turned ON'][aplnc])).total_seconds(),60)
                n = logs_df['Name'][aplnc]
                print(f'The {n} ran for a total of {int(mm)} minutes and {int(ss)} seconds')
                break #breaks i iterator creating error because never ending loop is occuring have to move this break out.
#         if m==1:
#             ser.close()
#             break
        

0.00,2020-11-5 4:42:34.736

27.00,2020-11-5 4:42:39.713

27.00,2020-11-5 4:42:42.201

An appliance  was detected at 2020-11-5 4:42:36.90
! Some people named it as ['ceilingfan'], would you like to name it?
fan
27.00,2020-11-5 4:42:47.177

 fan is being tested for being turned OFF
306.00,2020-11-5 4:42:52.155

306.00,2020-11-5 4:42:54.644

An appliance  was detected at 2020-11-5 4:42:49.918
! Some people named it as ['washing machine'], would you like to name it?
heater
305.00,2020-11-5 4:42:59.623

 fan is being tested for being turned OFF
number of active devices excluding device no. 0are 1
Cumulative Rise = 279.0
value not same
value not same
 heater is being tested for being turned OFF
number of active devices excluding device no. 1are 1
Cumulative Rise = 27.066666666666666
value not same
value not same
304.00,2020-11-5 4:43:4.601

 fan is being tested for being turned OFF
number of active devices excluding device no. 0are 1
Cumulative Rise = 279.0
value not same
value not same
 hea

KeyboardInterrupt: 

In [739]:
ser.close()

In [740]:
logs_df

,Sig,Name,Turned ON,Init Rise,Total Rise,Fall,Turned OFF,Flag
0,"[[0.06666666666666667, 27.066666666666666, 27....",fan,2020-11-05 04:42:36.900,27.066667,-0.066667,-0.066667,2020-11-05 04:43:34.462000,1
1,"[[4.0, 232.0, 232.0, 232.0, 232.0, 232.0, 232....",heater,2020-11-05 04:42:49.918,279.000000,27.105556,27.000000,2020-11-05 04:43:19.536000,1


In [714]:
active_appliances

2

In [61]:
import datetime

In [69]:
pd.to_datetime('2020-11-03 00:28:50.575000') + datetime.timedelta(seconds=3)

Timestamp('2020-11-03 00:28:53.575000')